In [1]:
import netCDF4 as nc
import numpy as np
import datetime
import xarray as xr
import pyart
import wradlib as wrl
from matplotlib import pyplot as plt

file = '/home/jovyan/work/project_0/sri_event1_test.nc'
#file = '/shared_space/project_0/radar/pro/precip.nc'
fp = xr.open_dataset(file)
totaltime = len(fp['time'])
totallon = len(fp['lat'])
totallat = len(fp['lon'])
data = np.array(fp['precip'][:,:,:])
times = np.array(fp['time'][:])
print('total time:  ', totaltime)

print('aqui passou 1 ')
## creating array to accumulation 30 minutes #############################
acum30 = []
time30 = []
i=1
j=4
while j <= totaltime:
    soma1 = data[:,:,i:j]
    print('30min   ',j)
    #print('soma1',soma1[400,346,:])
    soma2 = np.sum(soma1,axis=2)
    #print('soma2',soma2[400,346])
    time30.append(times[j-1])
    acum30.append(soma2)
    i=j
    j=j+3
acum30 = np.array(acum30)
print('aqui passou 2 ')
## creating array to accumulation 1h #####################################
acum1h = []
time1h = []
i=1
j=7
while j <= totaltime:
    soma1 = data[:,:,i:j]
    print('1h   ',j)
    soma2 = np.sum(soma1,axis=2)
    acum1h.append(soma2)
    time1h.append(times[j-1])
    i=j
    j=j+6
acum1h = np.array(acum1h)
print('aqui passou 3 ')
## creating array to accumulation 24h #####################################
acum24h = []
time24h = []
i=1
j=7
while j <= totaltime:
    soma1 = data[:,:,i:j]
    print('24h   ',j)
    soma2 = np.sum(soma1,axis=2)
    acum24h.append(soma2)
    time24h.append(times[j-1])
    i=j
    j=j+6
acum1h = np.array(acum24h)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

total time:   18
aqui passou 1 
30min    4
30min    7
30min    10
30min    13
30min    16
aqui passou 2 
1h    7
1h    13
aqui passou 3 
24h    7
24h    13


In [79]:
print(np.shape(acum30)[0])
print(np.shape(acum1h))
print(time30)
print(time1h)

5
(2, 999, 1008)
[20190616003, 20190616010, 20190616013, 20190616020, 20190616023]
[20190616010, 20190616020]


In [2]:
import netCDF4 as nc
import numpy as np
import datetime
import glob
import os
import numba

#Let's create the output netcdf file first for 30 minutes ##########################################
file = 'pre_acum30_sri.nc'
os.system('rm %s' % file)
fp = nc.Dataset(file,'w')
#n_time=432
n_time=np.shape(acum30)[0]
#Create dimensions
fp.createDimension("time", n_time)
fp.createDimension("lat", 999)
fp.createDimension("lon", 1008)
##Create variables
#coordinates
a_lon=np.array(range(1008))
a_lat=np.array(range(999))
delta_lon=a_lon*0.004534396945
delta_lat=a_lat*0.004534396945
lon=delta_lon+(-10.455523299458)
lat=delta_lat+(-37.743348264453)
#time
time=time30
fp.createVariable("time","i8",("time",))
fp.createVariable("lat","f8",("lat",))
fp.createVariable("lon","f8",("lon",))
fp.createVariable("precip","f4",("time","lat","lon"))

fp['lat'][:] = lat[:]
fp['lon'][:] = lon[:]
fp['time'][:] = time[0:n_time]
fp['precip'][:,:,:] = acum30[:,:,:]

fp['precip'].units = "mm/30min" ;
fp['precip'].description = "accumulated rainfall for 30 minutes intervals - weather radar" ;

#Close the file
fp.close()

In [51]:
#print(data[400,1:1000,1])

#print(totaltime)
#print(np.shape(soma2))
#print(np.shape(soma1))
#print(np.shape(acum30))

#print(data[400,346,1:18])
#print(data[400,346,1])
#print(data[400,346,2])
#print(data[400,346,3])
#print(data[400,346,4])
#print(data[400,346,5])
#print(data[400,346,6])
#print(data[400,346,7])


#print(acum30[:,400,346])
#print(acum1h[:,400,346])
#print(soma2[400,346])

#plt.figure(figsize=(10,10))
#plt.subplot(4,1,1)
#plt.imshow(data[:,:,0],cmap=plt.get_cmap('RdBu_r'))
#plt.subplot(4,1,2)
#plt.imshow(soma2[:,:],cmap=plt.get_cmap('RdBu_r'))
#cb = plt.colorbar(orientation='horizontal',shrink=0.8,pad=0.05)
#cb.ax.tick_params(labelsize=15)
#cb.set_label('Monthly temperature ($^o$C)',fontsize=20)
#plt.show()

In [ ]:
acum1h = []
## creating array to accumulation 30 minutes 
i=1
j=4
while j <= totaltime:
    soma1 = data[:,:,i:j]
    soma2 = np.sum(soma1,axis=2)
    acum1h.append(soma2)
    i=j
    j=j+3
acum1h = np.array(acum1h)

In [ ]:
filename = '/home/jovyan/work/project_0/pre_test_onefile.nc'

radar = pyart.io.read_grid(filename)
display = pyart.graph.RadarDisplay(radar)
fig = plt.figure()
ax = fig.add_subplot(111)
display.plot('precip', 0, vmin=-32, vmax=64.)
display.plot_range_rings([10, 20, 30, 40])
#display.plot_cross_hair(5.)
plt.show()